In [161]:
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup
from lxml import etree
import requests
import pandas as pd

In [198]:
screeners = [
    "most_actives",
    "day_losers",
    "aggressive_small_caps",
    "day_gainers"
    ]

In [184]:
URL = "https://finance.yahoo.com/screener/predefined/aggressive_small_caps?count=100&offset=0"

HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \
            (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',\
            'Accept-Language': 'en-US, en;q=0.5'})
  
webpage = requests.get(URL, headers=HEADERS)
soup = BeautifulSoup(webpage.content, "html.parser")
dom = etree.HTML(str(soup))

In [185]:
tickers = []
stockNames = []
closingPrice = []
percentage = []
volumes = []
threeMonthVolume = []
marketCap = []
PERatio = []


# Stock names
for i in range(1,101):
    try:
        stockNames.append(dom.xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{i}]/td[2]/text()')[0])
    except Exception as e:
        stockNames.append(None)
        
#tickers
for i in range(1,101):
    try:
        tickers.append(dom.xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{i}]/td[1]/a')[0].text)
    except Exception as e:
        tickers.append(None)

#closing price
for i in range(1,101):
    try:
        closingPrice.append(dom.xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{i}]/td[3]/span')[0].text)
    except Exception as e:
        closingPrice.append(None)

#percentage
for i in range(1,101):
    try:
        percentage.append(dom.xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{i}]/td[5]/span')[0].text)
    except Exception as e:
        percentage.append(None)
        
#volumes
for i in range(1,101):
    try:
        volumes.append(dom.xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{i}]/td[6]/span')[0].text)
    except Exception as e:
        volumes.append(None)

#3M Volume
for i in range(1,101):
    try:
        threeMonthVolume.append(dom.xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{i}]/td[7]/text()')[0])
    except Exception as e:
        threeMonthVolume.append(None)

# Market Cap
for i in range(1,101):
    try:
        marketCap.append(dom.xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{i}]/td[8]/span')[0].text)
    except Exception as e:
        marketCap.append(None)
        
# P/E RATIO
for i in range(1,101):
    try:
        PERatio.append(dom.xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{i}]/td[9]/text()')[0])
    except Exception as e:
        PERatio.append(None)

In [199]:
df=pd.DataFrame()
df['tickers'] = tickers 
df['company'] = stockNames 
df['closing price'] = closingPrice 
df['percentage'] = percentage 
df['volume'] = volumes 
df['3M avg volume'] = threeMonthVolume
df['market cap'] = marketCap 
df['PE ratio'] = PERatio 

In [200]:
df = df.dropna(axis=0, subset=['tickers', 'company'])

In [201]:
#Volume, 3M Volume, Market cap

#text preprocessing 1
for i in range(len(df)):
    df['volume'].iloc[i] = df.volume.iloc[i].replace(",","")
    df['3M avg volume'].iloc[i] = df['3M avg volume'].iloc[i].replace(",","")
    df['market cap'].iloc[i] = df['market cap'].iloc[i].replace(",","")
    df.percentage.iloc[i] = float(df.percentage.iloc[i].replace("+","").replace("%",""))
    
#text preprocessing 2
df['volume'] = df['volume'].replace({'K': '*1e3', 'M': '*1e6','B':'*1e9'}, regex=True).map(pd.eval).astype(float)
df['3M avg volume'] = df['3M avg volume'].replace({'K': '*1e3', 'M': '*1e6','B':'*1e9'}, regex=True).map(pd.eval).astype(float)
df['market cap'] = df['market cap'].replace({'K': '*1e3', 'M': '*1e6','B':'*1e9'}, regex=True).map(pd.eval).astype(float)

#text preprocessing 3
df['PE ratio'] = df['PE ratio'].fillna(0).astype(float)

df.percentage = df.percentage.astype(float)

In [202]:
df.dtypes

tickers           object
company           object
closing price     object
percentage       float64
volume           float64
3M avg volume    float64
market cap       float64
PE ratio         float64
dtype: object

In [203]:
df

,tickers,company,closing price,percentage,volume,3M avg volume,market cap,PE ratio
0,OAS,Oasis Petroleum Inc.,72.46,0.75,139236.0,240845.0,1.476000e+09,0.00
1,AHT,"Ashford Hospitality Trust, Inc.",1.8800,-2.08,12556000.0,10198000.0,2.467260e+08,0.00
2,XL,XL Fleet Corp.,7.14,6.57,9051000.0,10893000.0,9.486990e+08,0.00
3,SEAC,"SeaChange International, Inc.",1.1600,6.42,7098000.0,11445000.0,4.958500e+07,0.00
4,RIDE,Lordstown Motors Corp.,9.94,2.37,3469000.0,9408000.0,1.610000e+09,0.00
...,...,...,...,...,...,...,...,...
95,COGT,"Cogent Biosciences, Inc.",8.67,-0.46,109290.0,418579.0,3.228470e+08,0.00
96,REPL,"Replimune Group, Inc.",31.51,3.14,315096.0,385788.0,1.380000e+09,0.00
97,APTS,"Preferred Apartment Communities, Inc.",10.46,2.15,308935.0,441087.0,5.309290e+08,0.00
98,OSG,"Overseas Shipholding Group, Inc.",2.2800,0.44,493794.0,680396.0,2.023920e+08,6.91
